# Mindnlp PEFT with DNA Language Models
This notebook demonstrates how to utilize parameter-efficient fine-tuning techniques (PEFT) from the PEFT library to fine-tune a DNA Language Model (DNA-LM). The fine-tuned DNA-LM will be applied to solve a task from the nucleotide benchmark dataset. Parameter-efficient fine-tuning (PEFT) techniques are crucial for adapting large pre-trained models to specific tasks with limited computational resources.

When using modelarts env, need to remove `RANK_TABLE_FILE` to use `MindSpore` dynamic graph mode.
```python
import os
if "RANK_TABLE_FILE" in os.environ:
    del os.environ["RANK_TABLE_FILE"]
```

## 1. Import relevant libraries
We'll start by importing the required libraries, including the mindnlp library and other dependencies.

In [4]:
import mindspore
import mindnlp
import numpy as np
import tqdm

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

## 2. Load models
We'll load a pre-trained DNA Language Model, "SpeciesLM", that serves as the base for fine-tuning. This is done using the transformers library from HuggingFace.

The tokenizer and the model comes from the paper, "Species-aware DNA language models capture regulatory elements and their evolution". Paper Link, Code Link. They introduce a species-aware DNA language model, which is trained on more than 800 species spanning over 500 million years of evolution.

In [5]:
from mindnlp.transformers import AutoTokenizer, AutoModelForMaskedLM

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision = "downstream_species_lm")
lm = AutoModelForMaskedLM.from_pretrained("gagneurlab/SpeciesLM", revision = "downstream_species_lm")

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [7]:
lm.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear (768 -> 768)
              (key): Linear (768 -> 768)
              (value): Linear (768 -> 768)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear (768 -> 768)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermedia

## 3. Prepare datasets
We'll load the `nucleotide_transformer_downstream_tasks` dataset, which contains 18 downstream tasks from the Nucleotide Transformer paper. This dataset provides a consistent genomics benchmark with binary classification tasks.

In [8]:
from mindnlp.dataset import load_dataset
raw_data = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks", "H3")
raw_data

{'train': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset at 0xfffe3ef8ac40>,
 'test': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset at 0xfffe3ef741f0>}

We'll use the "H3" subset of this dataset, which contains a total of 13,468 rows in the training data, and 1497 rows in the test data.

In [9]:
train_dataset = raw_data['train']
test_dataset = raw_data['test']

def show_dataset_info(dataset):
    print("dataset column: {}".format(dataset.get_col_names()))
    print("dataset size: {}".format(dataset.get_dataset_size()))
    print("dataset batch size: {}\n".format(dataset.get_batch_size()))

print("train dataset info:")
show_dataset_info(train_dataset)
print("test dataset info:")
show_dataset_info(test_dataset)

train dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 13468
dataset batch size: 1

test dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 1497
dataset batch size: 1



The dataset consists of three columns, `sequence`, `name` and `label`. An row in this dataset looks like:

In [10]:
for data in train_dataset:
    print(data)
    break

[Tensor(shape=[], dtype=String, value= 'TCACTTCGATTATTGAGGCAGTCTTCATTAAAGTTTATTACAATGGATATGGTATCACCAGTCTTGAACCTACAATCATCTATTTTAGGTGAGCTCGTAGGCATTATTGGAAAAGTGTTCTTTCTCTTAATAGAAGAGATTAAATACCCGATAATCACACCCAAAATTATTGTGGATGCCCAGATATCTTCTTGGTCATTGTTTTTTTTCGCTTCAATCTGTAATCTCTCTGCAAAATTTCGGGAGCCAATAGTGACAACATCGTCAATAATAAGTTTGATGGAATCGGAAAAAGATCTTAAAAATGTAAATGAGTATTTCCAAATAATGGCCAAAATGCTCTTTATATTGGAAAATAAAATAGTTGTTTCGCTCTTCGTAGTATTTAACATTTCCGTTCTTATCATTGTAAAGTCTGAGCCATATTCATATGGAAAAGTGCTTTTTAAACCTAGTTCCTCCATATTTTAGTTTTTTATCGATATTGGAAAAAAAAGAGC'), Tensor(shape=[], dtype=String, value= 'YBR063C_YBR063C_367930|0'), Tensor(shape=[], dtype=Int64, value= 0)]


We split out dataset into training, test, and validation sets.

In [11]:
train_dataset, valid_dataset = train_dataset.split([0.85, 0.15])

print("train dataset info:")
show_dataset_info(train_dataset)
print("valid dataset info:")
show_dataset_info(valid_dataset)

train dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 11448
dataset batch size: 1

valid dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 2020
dataset batch size: 1



Then, we use the tokenizer and a utility function we created, `get_kmers` to generate the final data and labels. The `get_kmers` function is essential for generating overlapping 6-mers needed by the language model (LM). By using k=6 and stride=1, we ensure that the model receives continuous and overlapping subsequences, capturing the local context within the biological sequence for more effective analysis and prediction.

In [12]:
def get_kmers(seq, k=6, stride=1):
    return [seq[i:i + k] for i in range(0, len(seq), stride) if i + k <= len(seq)]

In [13]:

BATCH_SIZE = 16
DATASET_LIMIT = 200 # NOTE: This dataset limit is set to 200, so that the training runs faster. It can be set to None to use the
                    # entire dataset

def process_sequence(data):
    sequence = data.tolist()
    sequence = "candida_glabrata " + " ".join(get_kmers(sequence))
    sequence = tokenizer(sequence)["input_ids"]
    return sequence

def nucleotide_dataset_process(dataset):
    # remove name column
    dataset = dataset.project(columns=['sequence', 'label'])

    # process sequence"""  """
    dataset = dataset.map(process_sequence, input_columns=['sequence'], output_columns=['sequence'])

    # change dataset size
    dataset = dataset.take(DATASET_LIMIT)

    # batch with padding
    dataset = dataset.padded_batch(batch_size=BATCH_SIZE,
                                   drop_remainder=True,
                                   pad_info={'sequence':(None, -100)})
    return dataset


Finally, we create a Dataset object for each our sets.

In [14]:
train_dataset = train_dataset.apply(nucleotide_dataset_process)
valid_dataset = valid_dataset.apply(nucleotide_dataset_process)
test_dataset = test_dataset.apply(nucleotide_dataset_process)

print("train dataset info:")
show_dataset_info(train_dataset)

print("valid dataset info:")
show_dataset_info(valid_dataset)

print("test dataset info:")
show_dataset_info(test_dataset)

for data in train_dataset:
    print(data)
    break

train dataset info:
dataset column: ['sequence', 'label']
dataset size: 12
dataset batch size: 16

valid dataset info:
dataset column: ['sequence', 'label']
dataset size: 12
dataset batch size: 16

test dataset info:
dataset column: ['sequence', 'label']
dataset size: 12
dataset batch size: 16

[Tensor(shape=[16, 498], dtype=Int64, value=
[[   2, 4724,  200 ... 3930, 3420,    3],
 [   2, 4724,  414 ...  518, 2057,    3],
 [   2, 4724,    8 ... 3674, 2394,    3],
 ...
 [   2, 4724, 1369 ... 3089,   54,    3],
 [   2, 4724, 3365 ... 2606, 2219,    3],
 [   2, 4724, 1630 ... 2727, 2702,    3]]), Tensor(shape=[16], dtype=Int64, value= [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1])]


## 4. Train model
Now, we'll train our DNA Language Model with the training dataset. We'll add a linear layer in the final layer of our language model, and then, train all the parameteres of our model with the training dataset.

In [15]:
import mindspore
from mindnlp.core import nn

class DNA_LM(nn.Module):
    def __init__(self, model, num_labels):
        super(DNA_LM, self).__init__()
        self.model = model.bert
        self.in_features = model.config.hidden_size
        self.out_features = num_labels
        self.classifier = nn.Linear(self.in_features, self.out_features)

    def forward(self, sequence, label=None):
        outputs = self.model(input_ids=sequence, attention_mask=None, output_hidden_states=True)
        sequence_output = outputs.hidden_states[-1]
        # Use the [CLS] token for classification
        cls_output = sequence_output[:, 0, :]
        logits = self.classifier(cls_output)

        loss = None
        if label is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.out_features), label.view(-1))

        return (loss, logits) if loss is not None else logits

# Number of classes for your classification task
num_labels = 2
classification_model = DNA_LM(lm, num_labels)
classification_model

DNA_LM(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear (768 -> 768)
              (key): Linear (768 -> 768)
              (value): Linear (768 -> 768)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear (768 -> 768)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
    

In [16]:
from mindnlp.engine import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_steps=1,
    logging_steps=1,
)

# Initialize Trainer
trainer = Trainer(
    model=classification_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

  0%|          | 0/60 [00:00<?, ?it/s]

  2%|▏         | 1/60 [00:22<21:42, 22.08s/it]

  2%|▏         | 1/60 [01:06<21:42, 22.08s/it]

{'loss': 0.8252, 'learning_rate': 1.9666666666666666e-05, 'epoch': 0.08}


  3%|▎         | 2/60 [01:07<34:40, 35.87s/it]

...

100%|██████████| 60/60 [02:47<00:00,  1.30it/s]

{'loss': 0.5196, 'learning_rate': 0.0, 'epoch': 5.0}


                                               
100%|██████████| 60/60 [02:51<00:00,  2.86s/it]

{'eval_loss': 0.5207817554473877, 'eval_runtime': 4.0536, 'eval_samples_per_second': 2.96, 'eval_steps_per_second': 0.493, 'epoch': 5.0}
{'train_runtime': 171.6193, 'train_samples_per_second': 5.594, 'train_steps_per_second': 0.35, 'train_loss': 0.5752782459060352, 'epoch': 5.0}


TrainOutput(global_step=60, training_loss=0.5752782459060352, metrics={'train_runtime': 171.6193, 'train_samples_per_second': 5.594, 'train_steps_per_second': 0.35, 'train_loss': 0.5752782459060352, 'epoch': 5.0})

## 5. Evaluation

In [17]:
# Generate predictions

predictions = trainer.predict(test_dataset)
logits = predictions.predictions
predicted_labels = logits.argmax(axis=-1)
print(predicted_labels)

100%|██████████| 12/12 [00:02<00:00,  4.35it/s]

[1 1 1 1 0 1 1 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 1 1 0 0 0 1
 1 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 1 0 1 1 0 0 0 1
 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 0 0 0 1
 1 0 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1
 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0
 1 1 0 1 1 0 1]


Then, we create a function to calculate the accuracy from the test and predicted labels.

In [24]:
def calculate_accuracy(true_labels, predicted_labels):

    assert len(true_labels) == len(predicted_labels), "Arrays must have the same length"
    correct_predictions = np.sum(true_labels == predicted_labels)
    accuracy = correct_predictions / len(true_labels)

    return accuracy

test_labels = []
for data in test_dataset:
    sequence, label = data
    for single_label in label:
        test_labels.append(single_label)

accuracy = calculate_accuracy(test_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.78


The results aren't that good, which we can attribute to the small dataset size.

## 6. Parameter Efficient Fine-Tuning Techniques
In this section, we demonstrate how to employ parameter-efficient fine-tuning (PEFT) techniques to adapt a pre-trained model for specific genomics tasks using the PEFT library.

The LoraConfig object is instantiated to configure the PEFT parameters:

+ task_type: Specifies the type of task, in this case, sequence classification (SEQ_CLS).
+ r: The rank of the LoRA matrices.
+ lora_alpha: Scaling factor for adaptive re-parameterization.
+ target_modules: Modules within the model to apply PEFT re-parameterization (query, key, value in this example).
+ lora_dropout: Dropout rate used during PEFT fine-tuning.

In [26]:
# Number of classes for your classification task
num_labels = 2
classification_model = DNA_LM(lm, num_labels)
classification_model

DNA_LM(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear (768 -> 768)
              (key): Linear (768 -> 768)
              (value): Linear (768 -> 768)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear (768 -> 768)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
    

In [27]:
from mindnlp.peft import LoraConfig, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0.01,
)

In [28]:
from mindnlp.peft import get_peft_model

peft_model = get_peft_model(classification_model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 442,368 || all params: 90,121,730 || trainable%: 0.49085608986866985


In [29]:
peft_model

PeftModel(
  (base_model): LoraModel(
    (model): DNA_LM(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(5504, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear (768 -> 768)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.01, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear (768 -> 8)
                    )
                    (lora_B): ModuleDict(

In [34]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_steps=1,
    logging_steps=1,
)

# Initialize Trainer
trainer = Trainer(
    model=peft_model.model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

  0%|          | 0/60 [00:00<?, ?it/s]

  2%|▏         | 1/60 [00:00<00:38,  1.54it/s]

{'loss': 0.865, 'learning_rate': 1.9666666666666666e-05, 'epoch': 0.08}


  3%|▎         | 2/60 [00:01<00:34,  1.66it/s]

{'loss': 0.8191, 'learning_rate': 1.9333333333333333e-05, 'epoch': 0.17}


...

100%|██████████| 60/60 [01:49<00:00,  1.50it/s]

{'loss': 0.8036, 'learning_rate': 0.0, 'epoch': 5.0}


                                               
100%|██████████| 60/60 [01:52<00:00,  1.88s/it]

{'eval_loss': 0.7075120806694031, 'eval_runtime': 3.9299, 'eval_samples_per_second': 3.054, 'eval_steps_per_second': 0.509, 'epoch': 5.0}
{'train_runtime': 112.9944, 'train_samples_per_second': 8.496, 'train_steps_per_second': 0.531, 'train_loss': 0.7251347482204438, 'epoch': 5.0}


TrainOutput(global_step=60, training_loss=0.7251347482204438, metrics={'train_runtime': 112.9944, 'train_samples_per_second': 8.496, 'train_steps_per_second': 0.531, 'train_loss': 0.7251347482204438, 'epoch': 5.0})

## 7. Evaluate PEFT Model

In [35]:
# Generate predictions

predictions = trainer.predict(test_dataset)
logits = predictions.predictions
predicted_labels = logits.argmax(axis=-1)
print(predicted_labels)

100%|██████████| 12/12 [00:02<00:00,  4.08it/s]

[0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0
 1 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0
 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0
 0 1 1 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1
 0 0 1 0 0 0 1 0 1 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0
 1 0 0 0 1 0 0]


In [36]:
def calculate_accuracy(true_labels, predicted_labels):

    assert len(true_labels) == len(predicted_labels), "Arrays must have the same length"
    correct_predictions = np.sum(true_labels == predicted_labels)
    accuracy = correct_predictions / len(true_labels)

    return accuracy

test_labels = []
for data in test_dataset:
    sequence, label = data
    for single_label in label:
        test_labels.append(single_label)

accuracy = calculate_accuracy(test_labels, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.58


As we can see, the PEFT model achieves similar performance to the baseline model, demonstrating the effectiveness of PEFT in adapting pre-trained models to specific tasks with limited computational resources.

With PEFT, we only train 442,368 parameters, which is 0.49% of the total parameters in the model. This is a significant reduction in computational resources compared to training the entire model from scratch.

We can improve the results by using a larger dataset, fine-tuning the model for more epochs or changing the hyperparameters (rank, learning rate, etc.).